In [1]:
import os
import json
import textract
import random

ModuleNotFoundError: No module named 'textract'

## 读取数据路径

In [2]:
# 读取数据的路径
def list_all_files(directory):
    # 路径合成
    file_paths = [os.path.join(directory, file) for file in os.listdir(directory)]
    # 返回结果
    return file_paths

In [15]:
# 示例用法
directory_path = '/gemini/data-1/Docs_10W/docs/docs/'  # 将这里的路径替换为你想要遍历的目录路径
file_paths = list_all_files(directory_path)

In [20]:
# 查看数据
file_paths[0]

'/gemini/data-1/Docs_10W/docs/docs/国民党军队政治工作转向因素试析（1945-1946）.doc'

## 保存路径

In [3]:
def save_to_json_file(data, file_path):
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        print(f"数据成功保存到 {file_path}")
    except Exception as e:
        print(f"保存文件时发生错误: {e}")    

In [25]:
# 使用函数保存数据到JSON文件
save_to_json_file(file_paths, '/gemini/code/Abstract_extract/ProcessedData/rdfybk/rdfybk_path.json')

数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/rdfybk_path.json


## 读取路径

In [4]:
def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"错误：找不到文件 {file_path}。请检查文件路径是否正确。")
        return None
    except json.JSONDecodeError:
        print(f"错误：无法解码文件 {file_path} 中的JSON数据。请确保这是一个有效的JSON文件。")
        return None

In [3]:
# 使用函数读取JSON文件
file_path = '/gemini/code/Abstract_extract/ProcessedData/rdfybk/rdfybk_path.json'  # 将这里的文件路径替换为你自己的JSON文件路径
file_paths = read_json_file(file_path)

## 读取doc文件测试

In [5]:
import subprocess

def extract_text_from_doc(doc_path):
    try:
        result = subprocess.run(['antiword', doc_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            return result.stdout.decode('utf-8')
        else:
            return None
    except Exception as e:
        return None

In [6]:
import textract

def extract_text_from_doc(doc_path):
    try:
        text = textract.process(doc_path, encoding='utf-8')
        return text.decode('utf-8')  # 返回解码后的字符串
    except Exception as e:
        return None

In [8]:
# 示例用法
text_content = extract_text_from_doc(file_paths[0])

if text_content is not None:
    print("文件内容如下：")
    print(len(text_content))

文件内容如下：
18141


## 批量读取doc文档内容

In [ ]:
# 抽取策略
# -一共65.7万篇文章，涉及类别244小类，以子类进行划分
# -我们预计按照比例进行抽取，其中抽取类型以大类比例进行统计，也就是每个大类需要抽取满相对应的数量
# -经过统计一共有287个小类，我们现在需要计算出每一个类别中对应的总数的数量
# -首先应该抽取出每一类下的文件名，每个文件名应该放在一个列表中
# -现在要根据列表进行数据的质检和合成，我们一=一共160个类，每个类仅获取50条数据即可
# -通过单体分析我们发现，内容提要后面一定需要着“内容提要：/n/n”代表了内容提要后面其实没有文本，所以我们可以以这个作为一个过滤要求
# -同时应该要求文本字数大于5000字以形成有效的学术全文本

In [7]:
import pandas as pd

# 替换为你的文件路径
file_path = '/gemini/code/Abstract_extract/ProcessedData/rdfybk/rdfybk.xlsx'

# 使用 pandas 读取 Excel 文件
rdfybk = pd.read_excel(file_path)

# 打印长度
print(f"全文本的长度为：{len(rdfybk)}")

全文本的长度为：656761


In [8]:
# 获取全文名称
rdfybk_file_name = list(rdfybk['file_name'])

In [4]:
# 统计分类的长度
rdfybk_fenlei = list(rdfybk['fenlei'])
set_rdfybk_fenlei = set(rdfybk_fenlei)
print(f"单一类别的数量为{len(set_rdfybk_fenlei)}")

单一类别的数量为287


In [5]:
# 统计每一个类别下的文本数量
file_count_by_type = rdfybk['fenlei'].value_counts().reset_index()
file_count_by_type

,fenlei,count
0,种植与养殖,44537
1,审计文摘,18600
2,金融与保险,15587
3,企业家信息,15272
4,中国共产党,11300
...,...,...
281,经济贸易,8
282,账务与会计导刊（理论版）,8
283,中学语文,5
284,经济学,2


In [20]:
# 找出来数量小于阈值的类别分界线
file_count_by_type_count = list(file_count_by_type['count'])
for idx, value in enumerate(file_count_by_type_count):
    if value < 1000:
        print(idx)
        break

160


In [36]:
# 现在要针对这160个类别，先构造一个类别对应的文本名称的字典
file_count_by_type_fenlei = list(file_count_by_type['fenlei'])
fenlei_filename_dicts = []
for value in file_count_by_type_fenlei[:160]:
    filtered_files = list(rdfybk.loc[rdfybk['fenlei'] == value, 'file_name'])
    fenlei_filename_dict = {}
    fenlei_filename_dict["type"] = value
    fenlei_filename_dict["type_list"] = filtered_files
    fenlei_filename_dicts.append(fenlei_filename_dict)

In [37]:
# 首先保存我们第一批得到的数据
save_to_json_file(fenlei_filename_dicts, '/gemini/code/Abstract_extract/ProcessedData/rdfybk/rdfybk_fenlei_filename.json')

数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/rdfybk_fenlei_filename.json


In [38]:
# 测试读取我们保存的第一批数据
fenlei_filename_dict = read_json_file('/gemini/code/Abstract_extract/ProcessedData/rdfybk/rdfybk_fenlei_filename.json')
print(type(fenlei_filename_dict))

<class 'list'>


In [46]:
fenlei_filename_dicts[1]['type']

'审计文摘'

In [61]:
directory_path = '/gemini/data-1/Docs_10W/docs/docs/'
doc_path = os.path.join(directory_path, fenlei_filename_dicts[10]['type_list'][2445])
text_content = extract_text_from_doc(doc_path)
print(text_content)

convert /gemini/data-1/Docs_10W/docs/docs/师德评价困境的成因及其超越.doc -> /tmp/tmpwwnzlhdz/师德评价困境的成因及其超越.txt using filter : Text (StarWriter/Web)
﻿
师德评价困境的成因及其超越
作 者：
苏启敏/刘晶  


作者简介：
苏启敏(1980- )，男，广东广州人，广州大学教育学院教授，教育学博士，主要从事教育哲学，教师教育研究，E-mail:sqm218@126.com；刘晶(1996- )，女，广东韶关人，广州大学教育学院硕士研究生，主要从事教师教育研究。广东 广州 510006



原文出处：
教师发展研究


内容提要：
师德评价能否突破当前所面临的种种困境，有赖于三个涉及道德哲学的基本问题能否得以解决：第一个是师德评价对象的问题。作为评价对象的师德需要区分其外部边界和内部边界，外部边界存在于公共道德、私人道德和专业道德之间；内部边界存在于底线师德和理想师德之间。第二个是师德评价目的的问题。有必要扬弃教师绩效问责目的和教师专业发展目的，进一步澄清学生向善发展目的。第三个问题是师德评价方式的问题。它包含师德评价的依据、程序和方法等方面，对它们的合理性及其限度展开论证，是师德评价成败的关键。


期刊代号：G1 
分类名称：教育学 
复印期号：2021 年 02 期 


关 键 词：
师德评价  道德哲学  底线师德  理想师  




       中图分类号：G451.6 文献标识码：A 文章编号：2096-319X(2020)03-0034-09
       对教师的评价通常包括德、能、勤、绩、廉五个部分，其中对师德的评价往往是人们最关注的领域。当前，从国家到各级教育行政管理部门和大中小学都高度强调师德的重要性，并且以评价作为规范教师道德行为的根本手段，甚至提出师德评价一票否决制等强硬措施。但因教师校外补课、不尊重或体罚学生、性侵或性骚扰学生等师德问题引发的矛盾纠纷依然频繁出现在公共舆论的风口浪尖，也引起了社会各界的强烈反响。令人困惑的是，教育界对师德评价不可谓不重视，相关研究文献和科研项目在数量上逐年增长，全国、各省市、各级各类学校的政策文本和规范性文件也不断出台，但师德评价的落实仍困难重重，其实效性也未能得到社会各界的认可。笔

In [64]:
text_content.split('内容提要：')[1].split('期刊代号：')[0].replace('\n','')

'师德评价能否突破当前所面临的种种困境，有赖于三个涉及道德哲学的基本问题能否得以解决：第一个是师德评价对象的问题。作为评价对象的师德需要区分其外部边界和内部边界，外部边界存在于公共道德、私人道德和专业道德之间；内部边界存在于底线师德和理想师德之间。第二个是师德评价目的的问题。有必要扬弃教师绩效问责目的和教师专业发展目的，进一步澄清学生向善发展目的。第三个问题是师德评价方式的问题。它包含师德评价的依据、程序和方法等方面，对它们的合理性及其限度展开论证，是师德评价成败的关键。'

In [11]:
# 直接对全文本进行无条件随机抽取，不再按照类被进行划分了
random.seed(42)
all_docs = []
num = 0
# 获取列表并打乱列表
shuffled_list = rdfybk_file_name[:]
random.shuffle(shuffled_list)
# 进行循环
for idx, doc in enumerate(shuffled_list[24507:]):
    # 构造路径
    directory_path = '/gemini/data-1/Docs_10W/docs/docs/'
    doc_path = os.path.join(directory_path, doc)
    # 读取并筛选出合适的文件
    text_content = extract_text_from_doc(doc_path)
    # 首先抽取出我们想要的内容
    try:
        abstract = text_content.split('内容提要：')[1].split('期刊代号：')[0].replace('\n','')
        content = text_content.split('\n\n\n\n\n')[1]
    except:
        print(f"数据提取失败{idx}")
        continue
    if text_content != None and len(abstract) > 30 and len(content) > 2000:
        doc_dict = {}
        doc_dict['title'] = doc
        doc_dict['abstract'] = abstract
        doc_dict['content'] = text_content
        num = num + 1
        all_docs.append(doc_dict)
        print(f"数据合格{idx}")
        save_to_json_file(all_docs, '/gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content4.json')
    else:
        print(f"原始数据长度{len(text_content)}, 摘要长度{len(abstract)},{idx}")
        pass
    # 退出机制
    if num >= 5000:
        break
# 9861 + 6651 + 7993

原始数据长度637, 摘要长度151,0
原始数据长度5059, 摘要长度0,1
数据合格2
数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content4.json
原始数据长度5486, 摘要长度0,3
数据合格4
数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content4.json
原始数据长度1067, 摘要长度0,5
数据提取失败6
数据合格7
数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content4.json
数据合格8
数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content4.json
原始数据长度7538, 摘要长度0,9
原始数据长度3540, 摘要长度0,10
数据合格11
数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content4.json
数据合格12
数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content4.json
原始数据长度3907, 摘要长度0,13
数据合格14
数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content4.json
数据合格15
数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content4.json
数据合格16
数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content4.json
原始数据长度1363, 摘要长度2,17
数据提取失败18
原始数据长度11241, 摘要长度0,19
数据合格20
数据成功保存到 /gemini/code/Abstract

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [66]:
random.seed(42)
all_docs = []
for fenlei_filename_dict in fenlei_filename_dicts[1:]:
    # 构造计数器
    num = 0
    # 获取列表并打乱列表
    docs_list = fenlei_filename_dict['type_list']
    shuffled_list = docs_list[:]
    random.shuffle(shuffled_list)
    # 进行循环
    for doc in shuffled_list:
        # 构造路径
        directory_path = '/gemini/data-1/Docs_10W/docs/docs/'
        doc_path = os.path.join(directory_path, doc)
        # 读取并筛选出合适的文件
        text_content = extract_text_from_doc(doc_path)
        # 首先抽取出我们想要的内容
        try:
            abstract = text_content.split('内容提要：')[1].split('期刊代号：')[0].replace('\n','')
            content = text_content.split('\n\n\n\n\n')[1]
        except:
            continue
        if text_content != None and len(abstract) > 30 and len(content) > 2000:
            doc_dict = {}
            doc_dict['title'] = doc
            doc_dict['abstract'] = abstract
            doc_dict['content'] = text_content
            num = num + 1
            all_docs.append(doc_dict)
            print("数据合格")
            save_to_json_file(all_docs, '/gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content.json')
        elif text_content == None:
            print("数据提取失败")
            pass
        else:
            print(len(abstract))
            pass
        # 退出机制
        if num >= 50:
            break

0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
2
0
0
0
0
0
0
0
2
0
0
2
0
0
0
2
0
2
0
0
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
数据合格
数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content.json
0
0
2
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
2
0
0
0
0
2
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
2
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
0
0
0
0
0
0
2
0
0
0
0
0
0
0
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0



KeyboardInterrupt

